In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import when,lit,col
from pyspark.sql.types import StringType

In [ ]:
spark = SparkSession \
.builder \
.appName("Python Spark create RDD example") \
.config("spark.some.config.option", "some-value") \
.getOrCreate()

In [10]:
url = 'https://www.bbc.com/sport/football/premier-league/table'
headers = []
page = requests.get(url)
soup = BeautifulSoup(page.text,  "html.parser")
table= soup.find("table", class_="ssrcss-14j0ip6-Table e3bga5w4")

for i in table.find_all('th'):
    title = i.text
    headers.append(title)
league_table = pd.DataFrame(columns = headers)
for j in table.find_all('tr')[1:]:
    row_data = j.find_all('td')
    row = [i.text for i in row_data]
    length = len(league_table)
    league_table.loc[length] = row

league_table

,Position,Team,Played,Won,Drawn,Lost,Goals For,Goals Against,Goal Difference,Points,"Form, Last 6 games, Oldest first"
0,1,Arsenal,17,14,2,1,40,14,26,44,WResult WinWResult WinWResult WinWResult WinWR...
1,2,Manchester City,16,11,3,2,44,16,28,36,WResult WinWResult WinWResult WinLResult LossW...
2,3,Newcastle United,18,9,8,1,32,11,21,35,WResult WinWResult WinWResult WinWResult WinDR...
3,4,Manchester United,17,11,2,4,27,20,7,35,WResult WinLResult LossWResult WinWResult WinW...
4,5,Tottenham Hotspur,18,10,3,5,37,25,12,33,WResult WinLResult LossWResult WinDResult Draw...
5,6,Liverpool,17,8,4,5,34,22,12,28,LResult LossWResult WinWResult WinWResult WinW...
6,7,Fulham,18,8,4,6,30,27,3,28,DResult DrawLResult LossLResult LossWResult Wi...
7,8,Brighton & Hove Albion,17,8,3,6,32,25,7,27,WResult WinWResult WinLResult LossWResult WinL...
8,9,Brentford,18,6,8,4,30,28,2,26,DResult DrawDResult DrawWResult WinDResult Dra...
9,10,Chelsea,16,7,4,5,20,18,2,25,DResult DrawLResult LossLResult LossLResult Lo...


Picked up JAVA_TOOL_OPTIONS:  -Xmx3489m
Picked up JAVA_TOOL_OPTIONS:  -Xmx3489m


23/01/05 00:56:24 WARN Utils: Your hostname, georgemicha-premierleag-b6ljki953qu resolves to a loopback address: 127.0.0.1; using 10.0.5.2 instead (on interface ceth0)
23/01/05 00:56:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/05 00:56:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## TRANSFORMATION WITH PYSPARK

In [24]:
#drop unwanted column
league = league.drop("Form, Last 6 games, Oldest first") 
#rename columns
league = league.withColumnRenamed("Goals For","Goals_For").withColumnRenamed("Goals Against","Goals_Against") \
                .withColumnRenamed("Goal Difference","Goal_Difference")
#add new columns based on existing columns
league = league.withColumn("grade_class",  when((league.Lost == 0) & (league.Drawn == 0) \
                     & (league.Points <= league['Goals_For']), lit("S"))
                    .when((league.Lost == 0) & (league.Drawn <= 2), lit("A+"))
                    .when((league.Lost <= 1) & (league.Drawn <= 3), lit("A"))
                   .when((league.Lost <= 1  ) & (league.Drawn <= 4  ), lit('B'))
                   .otherwise('C')) 

league = league.withColumn('conqueror', lit(league.Points - (league.Played * 3)).cast('integer'))
# conqueror column, the difference between the teams current points and the points if they didn't loose or draw any math
league.show()

+--------+--------------------+------+---+-----+----+---------+-------------+---------------+------+-----------+---------+
|Position|                Team|Played|Won|Drawn|Lost|Goals_For|Goals_Against|Goal_Difference|Points|grade_class|conqueror|
+--------+--------------------+------+---+-----+----+---------+-------------+---------------+------+-----------+---------+
|       1|             Arsenal|    17| 14|    2|   1|       40|           14|             26|    44|          A|       -7|
|       2|     Manchester City|    16| 11|    3|   2|       44|           16|             28|    36|          C|      -12|
|       3|    Newcastle United|    18|  9|    8|   1|       32|           11|             21|    35|          C|      -19|
|       4|   Manchester United|    17| 11|    2|   4|       27|           20|              7|    35|          C|      -16|
|       5|   Tottenham Hotspur|    18| 10|    3|   5|       37|           25|             12|    33|          C|      -21|
|       6|      